In [22]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# Requirements: ase, numpy, pandas

import os, re, json, glob
import numpy as np
import pandas as pd
from ase.io import read
from ase.data import atomic_masses, chemical_symbols

In [23]:
# =========================
# ========= Config =========
# =========================
root_dir        = "./Normal_300K_01eV/2/"                 # dossier racine
mode            = "lammps"                 # "vasp" | "lammps" | "xyz"
xyz_path        = "traj.xyz"               # si mode == "xyz"
pattern_vasp    = "vasprun-*_full.xml"
pattern_lmp     = "nve_*.lammpstrj"

# Diatomique cible
# Priorité de sélection par frame: index explicite > id > type > symboles > heuristique
A_index = None
B_index = None

A_id    = None
B_id    = None

# Mapping LAMMPS: 1->N, 2->O, 3->C par exemple
lmp_type_to_symbol = {1: "N", 2: "O", 3: "C"}
A_type, B_type = 1, 2

A_symbol = "N"
B_symbol = "O"

# Fenêtre pour détecter la liaison si besoin
re_A  = 1.150
bond_guess_window = (0.8, 1.6)

# Référence surface
surf_normal = np.array([0, 0, 1.0])
zref_mode   = "constant"           # "constant" | "avg_layer_indices"
zref_const  = 0.0
zref_indices = []

# Seuil scattering
scatter_Z_thr_A = 8.0

# Paramètres Morse
De_eV  = 6.495
a_invA = 2.30
# re_A défini plus haut

# Pas de temps si vitesses absentes
dt_fs = 1.0

# Masses override si besoin
mA_amu_override = None
mB_amu_override = None

In [24]:
# =========================
# ======= Constantes ======
# =========================
eV_to_J   = 1.602176634e-19
J_to_eV   = 1.0 / eV_to_J
amu_to_kg = 1.66053906660e-27
Ang_to_m  = 1.0e-10
fs_to_s   = 1.0e-15
hbar_Js   = 1.054571817e-34

In [25]:
# =========================
# ====== Utilitaires ======
# =========================
def unit_vec(v):
    n = np.linalg.norm(v)
    return v / n if n > 0 else np.zeros_like(v)

def mic_dr(rB, rA, cell=None, pbc=None):
    dr = rB - rA
    if cell is None or pbc is None or not any(pbc):
        return dr
    cell = np.array(cell)
    invc = np.linalg.inv(cell.T)
    s = dr @ invc
    s -= np.round(s) * np.array(pbc, int)
    return s @ cell.T

def parse_sorted(files, regex_int):
    def extract(x):
        m = re.search(regex_int, os.path.basename(x))
        return int(m.group(1)) if m else -1
    return sorted(files, key=extract)

def get_images():
    if mode == "vasp":
        files = glob.glob(os.path.join(root_dir, pattern_vasp))
        files = parse_sorted(files, r"vasprun-(\d+)_full\.xml")
        imgs = []
        for f in files:
            part = read(f, index=":", format="vasp-xml")
            imgs.extend(part if isinstance(part, list) else [part])
        return imgs
    elif mode == "lammps":
        files = glob.glob(os.path.join(root_dir, pattern_lmp))
        files = parse_sorted(files, r"nve_(\d+)\.lammpstrj")
        imgs = []
        for f in files:
            try:
                part = read(f, index=":", format="lammps-dump-text")
            except Exception:
                part = read(f, index=":")
            imgs.extend(part if isinstance(part, list) else [part])
        return imgs
    elif mode == "xyz":
        part = read(os.path.join(root_dir, xyz_path), index=":", format="xyz")
        return part if isinstance(part, list) else [part]
    else:
        raise ValueError("mode inconnu")

def try_velocities(images, iA, iB):
    VA, VB = [], []
    ok = True
    for im in images:
        v = None
        arr = getattr(im, "arrays", {})
        if isinstance(arr, dict):
            v = arr.get("velocities", arr.get("velocity", None))
            if v is None and "momenta" in arr:
                m = im.get_masses()[:, None]
                v = arr["momenta"] / m
        if v is None:
            try:
                v = im.get_velocities()
            except Exception:
                v = None
        if v is None:
            ok = False
            break
        VA.append(v[iA]); VB.append(v[iB])
    if not ok:
        return None, None, False
    VA = np.array(VA) * (Ang_to_m / fs_to_s)
    VB = np.array(VB) * (Ang_to_m / fs_to_s)
    return VA, VB, True

def estimate_vel_local(prev_pos, next_pos, dt_fs):
    dt = dt_fs * fs_to_s
    return ((next_pos - prev_pos) * Ang_to_m) / (2*dt)

def get_zref(positions_A, mode, const, idxs):
    if mode == "constant":
        return const
    if mode == "avg_layer_indices" and idxs:
        return float(np.mean(positions_A[idxs, 2]))
    return 0.0

def morse_V_eV(r_A, De, a, re):
    return De * (1.0 - np.exp(-a * (r_A - re)))**2

In [26]:
# =========================
# === Géométrie/Angles ====
# =========================
def geom_angles_and_dist(rA_A, rB_A, cell, pbc, surf_n, positions_frame,
                         zref_mode, zref_const, zref_indices, mA, mB):
    dr_A = mic_dr(rB_A, rA_A, cell, pbc)
    r = np.linalg.norm(dr_A)
    rhat = unit_vec(dr_A)
    Rcm_m = (mA*(rA_A*Ang_to_m) + mB*(rB_A*Ang_to_m)) / (mA + mB)
    Rcm_A = Rcm_m / Ang_to_m
    zref = get_zref(positions_frame, zref_mode, zref_const, zref_indices)
    zdist = Rcm_A[2] - zref
    theta_m = np.degrees(np.arccos(np.clip(rhat[2], -1.0, 1.0)))
    phi_m   = np.degrees(np.arctan2(rhat[1], rhat[0]))
    return r, rhat, Rcm_A, zdist, theta_m, phi_m

def velocity_angles(Vcm_mps, surf_n):
    vn = unit_vec(Vcm_mps)
    th = np.degrees(np.arccos(np.clip(np.dot(vn, unit_vec(surf_n)), -1.0, 1.0))) if np.linalg.norm(Vcm_mps)>0 else 0.0
    phi = np.degrees(np.arctan2(Vcm_mps[1], Vcm_mps[0]))
    return th, phi, float(np.linalg.norm(Vcm_mps))

In [27]:
# =========================
# ======== Énergies =======
# =========================
def energies_diatomic(dr_A, vA_mps, vB_mps, mA, mB, De_eV, a_invA, re_A):
    M  = mA + mB
    mu = mA*mB/M
    r_m = dr_A * Ang_to_m
    r2  = float(np.dot(r_m, r_m))
    KE_J = 0.5*mA*np.dot(vA_mps, vA_mps) + 0.5*mB*np.dot(vB_mps, vB_mps)
    KE_eV = KE_J * J_to_eV
    vrel = vB_mps - vA_mps
    cx = np.cross(r_m, vrel)
    Erot_J = 0.0 if r2 == 0.0 else 0.5 * mu * (np.dot(cx, cx)/r2)
    Erot_eV = Erot_J * J_to_eV
    L_mag = mu * np.linalg.norm(cx)
    j_cont = L_mag / hbar_Js
    j_level = int(np.rint(j_cont)) if np.isfinite(j_cont) else 0
    j_level = max(j_level, 0)
    rhat = unit_vec(r_m)
    v_rad = float(np.dot(vrel, rhat))
    Evib_J = 0.5*mu*(v_rad**2) + morse_V_eV(np.linalg.norm(dr_A), De_eV, a_invA, re_A)*eV_to_J
    Evib_eV = Evib_J * J_to_eV
    return KE_eV, Erot_eV, j_level, Evib_eV

# =========================
# === Sélecteur A,B frame ==
# =========================
def find_AB_indices_frame(im, re_A, win, A_index=None, B_index=None,
                          A_id=None, B_id=None,
                          A_type=None, B_type=None,
                          A_symbol="N", B_symbol="O"):
    pos = im.positions
    # indices explicites
    if A_index is not None and B_index is not None:
        return int(A_index), int(B_index)
    # via id LAMMPS
    ids = im.arrays.get('id', None)
    if ids is not None and A_id is not None and B_id is not None:
        id_to_idx = {int(i): k for k, i in enumerate(ids)}
        return id_to_idx[int(A_id)], id_to_idx[int(B_id)]
    # via type LAMMPS
    types = im.arrays.get('type', None)
    if types is not None and A_type is not None and B_type is not None:
        idxA = np.where(types == int(A_type))[0]
        idxB = np.where(types == int(B_type))[0]
        best = None; best_err = np.inf
        for a in idxA:
            for b in idxB:
                r = np.linalg.norm(mic_dr(pos[b], pos[a], im.cell, im.pbc))
                err = abs(r - re_A)
                if win[0] <= r <= win[1] and err < best_err:
                    best = (a, b); best_err = err
        if best is not None:
            return best
    # via symboles
    try:
        syms = im.get_chemical_symbols()
        idxA = [i for i,s in enumerate(syms) if s == A_symbol]
        idxB = [i for i,s in enumerate(syms) if s == B_symbol]
        best = None; best_err = np.inf
        for a in idxA:
            for b in idxB:
                r = np.linalg.norm(mic_dr(pos[b], pos[a], im.cell, im.pbc))
                err = abs(r - re_A)
                if win[0] <= r <= win[1] and err < best_err:
                    best = (a, b); best_err = err
        if best is not None:
            return best
    except Exception:
        pass
    # heuristique
    n = len(pos)
    best = None; best_err = np.inf
    for a in range(n):
        for b in range(a+1, n):
            r = np.linalg.norm(mic_dr(pos[b], pos[a], im.cell, im.pbc))
            err = abs(r - re_A)
            if win[0] <= r <= win[1] and err < best_err:
                best = (a, b); best_err = err
    if best is None:
        raise RuntimeError("NO introuvable pour cette frame. Ajuste bond_guess_window ou fournis A_index B_index.")
    return best

In [28]:
# =========================
# ========= Main ==========
# =========================
images = get_images()
if not images:
    raise RuntimeError("Aucune image lue.")

# Masses
if mA_amu_override is not None and mB_amu_override is not None:
    mA_amu = float(mA_amu_override); mB_amu = float(mB_amu_override)
else:
    mA_amu = atomic_masses[chemical_symbols.index(A_symbol)]
    mB_amu = atomic_masses[chemical_symbols.index(B_symbol)]
mA = mA_amu * amu_to_kg
mB = mB_amu * amu_to_kg
M  = mA + mB

rows = []
ever_scattered = 0
surf_n = unit_vec(surf_normal)

for t, im in enumerate(images):
    # sélection A,B par frame
    iA, iB = find_AB_indices_frame(
        im, re_A, bond_guess_window,
        A_index=A_index, B_index=B_index,
        A_id=A_id, B_id=B_id,
        A_type=A_type, B_type=B_type,
        A_symbol=A_symbol, B_symbol=B_symbol
    )

    rA_A = im.positions[iA]
    rB_A = im.positions[iB]

    # vitesses A,B
    vA_mps = vB_mps = None
    arr = getattr(im, "arrays", {})
    v = arr.get("velocities", arr.get("velocity", None))
    if v is None:
        try:
            v = im.get_velocities()
        except Exception:
            v = None
    if v is not None:
        vA_mps = v[iA] * (Ang_to_m / fs_to_s)
        vB_mps = v[iB] * (Ang_to_m / fs_to_s)
    else:
        if 0 < t < len(images)-1:
            vA_mps = estimate_vel_local(images[t-1].positions[iA], images[t+1].positions[iA], dt_fs)
            vB_mps = estimate_vel_local(images[t-1].positions[iB], images[t+1].positions[iB], dt_fs)
        elif t == 0 and len(images) > 1:
            dt = dt_fs * fs_to_s
            vA_mps = ((images[1].positions[iA] - rA_A) * Ang_to_m) / dt
            vB_mps = ((images[1].positions[iB] - rB_A) * Ang_to_m) / dt
        elif t == len(images)-1 and len(images) > 1:
            dt = dt_fs * fs_to_s
            vA_mps = ((rA_A - images[t-1].positions[iA]) * Ang_to_m) / dt
            vB_mps = ((rB_A - images[t-1].positions[iB]) * Ang_to_m) / dt
        else:
            raise RuntimeError("Vitesses indisponibles et pas assez de frames pour dériver.")

    # géométrie
    r_NO, rhat, Rcm_A, zdist, theta_m, phi_m = geom_angles_and_dist(
        rA_A, rB_A, im.cell, im.pbc, surf_n, im.positions,
        zref_mode, zref_const, zref_indices, mA, mB
    )

    # vitesse COM et angles
    Vcm = (mA*vA_mps + mB*vB_mps) / M
    theta_v, phi_v, Vcm_norm = velocity_angles(Vcm, surf_n)

    # énergies
    dr_A = mic_dr(rB_A, rA_A, im.cell, im.pbc)
    KE_eV, Erot_eV, j_level, Evib_eV = energies_diatomic(
        dr_A, vA_mps, vB_mps, mA, mB, De_eV, a_invA, re_A
    )

    scattered = 1 if zdist > scatter_Z_thr_A else 0
    ever_scattered = max(ever_scattered, scattered)

    rows.append(dict(
        frame=t,
        iA=iA, iB=iB,
        r_NO_A=r_NO,
        z_COM_minus_zref_A=zdist,
        scattered=scattered,
        theta_m_deg=theta_m, phi_m_deg=phi_m,
        theta_Vcm_deg=theta_v, phi_Vcm_deg=phi_v,
        Vcm_norm_mps=Vcm_norm,
        KE_total_eV=KE_eV, E_rot_eV=Erot_eV, j_rot=j_level, E_vib_eV=Evib_eV
    ))

# sorties
df = pd.DataFrame(rows)
df.to_csv("per_frame_diatomic.csv", index=False)

summary = dict(
    n_frames=int(len(images)),
    mode=mode,
    patterns=dict(vasp=pattern_vasp, lammps=pattern_lmp, xyz=xyz_path),
    A_symbol=A_symbol, B_symbol=B_symbol,
    masses_amu=dict(A=float(mA_amu), B=float(mB_amu)),
    scatter_threshold_A=float(scatter_Z_thr_A),
    ever_scattered=int(ever_scattered),
    morse=dict(De_eV=float(De_eV), a_invA=float(a_invA), re_A=float(re_A)),
    zref_mode=zref_mode
)
with open("summary.json","w") as f:
    json.dump(summary, f, indent=2)

print("OK -> per_frame_diatomic.csv, summary.json")

OK -> per_frame_diatomic.csv, summary.json
